# Assess fairness and accuracy of classifiers

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pickle
import torch

from torchmetrics import AUROC, F1Score, Accuracy, MeanSquaredError
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from modules.helpers import seed
from modules import evaluation
from modules.predictors_modules import StandardClf, FairClf, FairClf_naive, Density_estimator, StandardRegressor, FairRegressor, FairClf_naive_lambda, FairClf_lambda

from modules import helpers

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

seed(1)

In [ ]:
model_location = "../models/"

# Evaluate model performance


#### a) Classifiers

In [ ]:
seed(1)

# propensity model:
label_col = "A"

experiment_list = ["sim_cont_d"]        # Here enter experiments to be evaluated
for experiment in experiment_list:

    data_location = "../data/simulator/"+ experiment + "_full_dataframe"   
    checkpointpath_density = model_location + "/density_estimator_A_" + experiment + "/density_estimator_A_" + experiment + "_checkpoints.ckpt"
    density_estimator = Density_estimator.load_from_checkpoint(checkpointpath_density)

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["Y", "M", "USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = density_estimator(covariates)
        auc = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC = " + str(auc))

In [ ]:
seed(1)

# density estimator:
label_col = "M"

experiment_list = ["sim_cont_e"]        # Here enter experiments to be evaluated

for experiment in experiment_list:

    data_location = "../data/simulator/"+ experiment + "_full_dataframe"  
    checkpointpath_density = model_location + "/density_estimator_" + experiment + "/density_estimator_" + experiment + "_checkpoints.ckpt"
    density_estimator = Density_estimator.load_from_checkpoint(checkpointpath_density)

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["Y", "USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = density_estimator(covariates)
        auc = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC = " + str(auc))

In [ ]:
seed(1)

# StandardClf:
label_col = "Y"
experiment_list = ["sim_cont_e"]        # Here enter experiments to be evaluated

for experiment in experiment_list:

    data_location = "../data/simulator/"+ experiment + "_full_dataframe"
    standard = StandardClf.load_from_checkpoint(model_location + "StandardClf_"+ experiment + "/StandardClf_"+ experiment + "_checkpoints.ckpt")

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)
    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)

    auroc = AUROC(task="binary")
    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = standard(covariates)
        auc = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC = " + str(auc))

In [ ]:
seed(1)

# FaiClf_naive:
label_col = "Y"
experiment_list = ["sim_cont_e"]        # Here enter experiments to be evaluated

for experiment in experiment_list:
    naive = FairClf_naive.load_from_checkpoint(model_location + "FairClf_naive_lambda"+ experiment + "/FairClf_naive_lambda"+ experiment + "_checkpoints.ckpt")
    data_location = "../data/simulator/"+ experiment + "_full_dataframe"

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = naive(covariates)
        auc = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC = " + str(auc))

In [ ]:
seed(1)

# FaiClf:
label_col = "Y"
experiment_list = ["sim9"]        # Here enter experiments to be evaluated

for experiment in experiment_list:
    fair = FairClf.load_from_checkpoint(model_location + "FairClf_"+ experiment + "/FairClf_"+ experiment + "_checkpoints.ckpt")
    data_location = "../data/simulator/"+ experiment + "_full_dataframe"

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = fair(covariates)
        auc = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC = " + str(auc))

#### b) Real-world regressors

In [ ]:
seed(1)

# density:
label_col = "M"

experiment_list = ["prison"]
for experiment in experiment_list:

    data_location = "../data/prison/prison_dataframe"    
    checkpointpath_density = model_location + "/density_estimator_" + experiment + "/density_estimator_" + experiment + "_checkpoints.ckpt"
    density_estimator = Density_estimator.load_from_checkpoint(checkpointpath_density)

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["Y", "USE", "UIE", "UDE"],axis=1)
    accuracy = Accuracy(task="multiclass", num_classes= len(full_data["M"].unique()), average='weighted')

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        logits, out = density_estimator(covariates)
        acc = accuracy(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("Accuracy = " + str(acc))

In [ ]:
seed(1)

# StandardRegressor:

label_col = "Y"
experiment_list = ["prison"]

for experiment in experiment_list:

    data_location = "../data/prison/prison_dataframe"    
    standard = StandardRegressor.load_from_checkpoint(model_location + "StandardRegressor_"+ experiment + "/StandardRegressor_"+ experiment + "_checkpoints.ckpt")

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    mse = MeanSquaredError()

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        out = standard(covariates)
        result = mse(out.squeeze(), torch.tensor(test_data[label_col].to_numpy()).long())
        print("MSE = " + str(result))

In [ ]:
seed(1)

# FairRegressor:
label_col = "Y"
experiment_list = ["prison_sensitivity_4"]

for experiment in experiment_list:

    data_location = "../data/prison/prison_dataframe"    
    fair = FairRegressor.load_from_checkpoint(model_location + "FairRegressor_"+ experiment + "/FairRegressor_"+ experiment + "_checkpoints.ckpt")

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    mse = MeanSquaredError()

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()
    
    with torch.no_grad():
        out = fair(covariates)
        result = mse(out.squeeze(), torch.tensor(test_data[label_col].to_numpy()).long())
        print("MSE = " + str(result))

### Calculate fairness utility

In [ ]:
# Experiment UDE

seed(1)

label_col = "Y"
experiment_list = ["sim9b", "sim9c", "sim9d", "sim9e"]

sensitivity_parameter_list = [2.0]
checkpoints = "_checkpoints.ckpt"

for experiment in experiment_list:
    standard = StandardClf.load_from_checkpoint(model_location + "StandardClf_"+ experiment + "/StandardClf_"+ experiment + "_checkpoints.ckpt")
    naive = FairClf_naive.load_from_checkpoint(model_location + "FairClf_naive_"+ experiment + "/FairClf_naive_"+ experiment + "_checkpoints.ckpt")
    fair = FairClf.load_from_checkpoint(model_location + "FairClf_"+ experiment + "/FairClf_"+ experiment + "_checkpoints.ckpt")
    data_location = "../data/simulator/"+ experiment + "_full_dataframe"

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()

    bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
    #print(bounds)

    with torch.no_grad():
        logits, out = standard(covariates)
        auc_standard = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_standard))

        logits, out = naive(covariates)
        auc_naive = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_naive))

        logits, out = fair(covariates)
        auc_fair = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC fair = " + str(auc_fair))

    bounds_standard = bounds.loc[bounds["Model"]=="StandardClf"]
    bounds_naive = bounds.loc[bounds["Model"]=="FairClf_naive"]
    bounds_fair = bounds.loc[bounds["Model"]=="FairClf"]
    
    utility_standard = 0.5*auc_standard - 0.5*(1/3*np.max(np.abs(bounds_standard[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["SE_ub", "SE_lb"]])))
    utility_naive = 0.5*auc_naive - 0.5*(1/3*np.max(np.abs(bounds_naive[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["SE_ub", "SE_lb"]])))
    utility_fair = 0.5*auc_fair - 0.5*(1/3*np.max(np.abs(bounds_fair[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["SE_ub", "SE_lb"]])))

    print("Utility standard = " + str(utility_standard))
    print("Utility naive = " + str(utility_naive))
    print("Utility fair = " + str(utility_fair))

In [ ]:
# Experiment UIE

seed(1)

label_col = "Y"
experiment_list = ["sim10b", "sim10c", "sim10d", "sim10e"]

sensitivity_parameter_list = [2.0]
checkpoints = "_checkpoints.ckpt"

for experiment in experiment_list:
    standard = StandardClf.load_from_checkpoint(model_location + "StandardClf_"+ experiment + "/StandardClf_"+ experiment + "_checkpoints.ckpt")
    naive = FairClf_naive.load_from_checkpoint(model_location + "FairClf_naive_"+ experiment + "/FairClf_naive_"+ experiment + "_checkpoints.ckpt")
    fair = FairClf.load_from_checkpoint(model_location + "FairClf_"+ experiment + "/FairClf_"+ experiment + "_checkpoints.ckpt")
    data_location = "../data/simulator/"+ experiment + "_full_dataframe"

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()

    bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
    #print(bounds)

    with torch.no_grad():
        logits, out = standard(covariates)
        auc_standard = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_standard))

        logits, out = naive(covariates)
        auc_naive = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_naive))

        logits, out = fair(covariates)
        auc_fair = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC fair = " + str(auc_fair))

    bounds_standard = bounds.loc[bounds["Model"]=="StandardClf"]
    bounds_naive = bounds.loc[bounds["Model"]=="FairClf_naive"]
    bounds_fair = bounds.loc[bounds["Model"]=="FairClf"]
    
    utility_standard = 0.5*auc_standard - 0.5*(1/3*np.max(np.abs(bounds_standard[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["SE_ub", "SE_lb"]])))
    utility_naive = 0.5*auc_naive - 0.5*(1/3*np.max(np.abs(bounds_naive[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["SE_ub", "SE_lb"]])))
    utility_fair = 0.5*auc_fair - 0.5*(1/3*np.max(np.abs(bounds_fair[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["SE_ub", "SE_lb"]])))

    print("Utility standard = " + str(utility_standard))
    print("Utility naive = " + str(utility_naive))
    print("Utility fair = " + str(utility_fair))

In [ ]:
# Experiment continuous Z

seed(1)

label_col = "Y"
experiment_list = ["sim_cont_b", "sim_cont_c", "sim_cont_d", "sim_cont_e"]

sensitivity_parameter_list = [2.0]
checkpoints = "_checkpoints-v3.ckpt"

for experiment in experiment_list:
    standard = StandardClf.load_from_checkpoint(model_location + "StandardClf_"+ experiment + "/StandardClf_"+ experiment + "_checkpoints-v3.ckpt")
    naive = FairClf_naive_lambda.load_from_checkpoint(model_location + "FairClf_naive_lambda"+ experiment + "/FairClf_naive_lambda"+ experiment + "_checkpoints-v3.ckpt")
    fair = FairClf_lambda.load_from_checkpoint(model_location + "FairClf_lambda_"+ experiment + "/FairClf_lambda_"+ experiment + "_checkpoints-v3.ckpt")
    data_location = "../data/simulator/"+ experiment + "_full_dataframe"

    with open(data_location, "rb") as input:
        full_data = pickle.load(input)

    full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
    auroc = AUROC(task="binary")

    test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
    covariates = test_data.iloc[:, test_data.columns != label_col].values
    covariates = torch.tensor(covariates).float()

    bounds, oracle = evaluation.calculate_bounds_continuous(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
    #print(bounds)

    with torch.no_grad():
        logits, out = standard(covariates)
        auc_standard = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_standard))

        logits, out = naive(covariates)
        auc_naive = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC naive = " + str(auc_naive))

        logits, out = fair(covariates)
        auc_fair = auroc(out, torch.tensor(test_data[label_col].to_numpy()).long())
        print("AUC fair = " + str(auc_fair))

    bounds_standard = bounds.loc[bounds["Model"]=="StandardClf"]
    bounds_naive = bounds.loc[bounds["Model"]=="FairClf_naive"]
    bounds_fair = bounds.loc[bounds["Model"]=="FairClf"]
    
    utility_standard = 0.5*auc_standard - 0.5*(1/3*np.max(np.abs(bounds_standard[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_standard[["SE_ub", "SE_lb"]])))
    utility_naive = 0.5*auc_naive - 0.5*(1/3*np.max(np.abs(bounds_naive[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_naive[["SE_ub", "SE_lb"]])))
    utility_fair = 0.5*auc_fair - 0.5*(1/3*np.max(np.abs(bounds_fair[["DE_ub", "DE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["IE_ub", "IE_lb"]])) + 1/3*np.max(np.abs(bounds_fair[["SE_ub", "SE_lb"]])))

    print("Utility standard = " + str(utility_standard))
    print("Utility naive = " + str(utility_naive))
    print("Utility fair = " + str(utility_fair))

### Calculate bounds

Mean + Std of bounds for seed = 0 to seed = 4

In [ ]:
with open("../results/bounds_list_over_seeds_sim9", "rb") as input:
    bounds_matrix = pickle.load(input)

In [ ]:
standard = bounds_matrix.loc[(bounds_matrix["Model"] == "StandardClf") & (bounds_matrix["Experiment"]=="sim9e")].drop(["Sensitivity parameter", "Model", "Experiment"], axis = 1)
naive = bounds_matrix.loc[(bounds_matrix["Model"] == "FairClf_naive") & (bounds_matrix["Experiment"]=="sim9e")].drop(["Sensitivity parameter", "Model", "Experiment"], axis = 1)
fair = bounds_matrix.loc[(bounds_matrix["Model"] == "FairClf") & (bounds_matrix["Experiment"]=="sim9e")].drop(["Sensitivity parameter", "Model", "Experiment"], axis = 1)

print("standard mean = " + str(standard.mean()))
print("standard std = " + str(standard.std()))

print("naive mean = " + str(naive.mean()))
print("naive std = " + str(naive.std()))

print("fair mean = " + str(fair.mean()))
print("fair std = " + str(fair.std()))

In [ ]:
experiment_list = ["sim9b", "sim9c", "sim9d", "sim9e"]
conf_level = np.array([1,2,3,4])

checkpoint_list = ["_checkpoints.ckpt", "_checkpoints-v1.ckpt", "_checkpoints-v2.ckpt", "_checkpoints-v3.ckpt", "_checkpoints-v4.ckpt"]
sensitivity_parameter_list = [2.0]


bounds_matrix = pd.DataFrame(["Sensitivity parameter ", "Model", "Experiment", "DE_ub", "DE_lb", "IE_ub", "IE_lb", "SE_ub", "SE_lb"])

for checkpoints in checkpoint_list:
    with torch.no_grad():   
        bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
        bounds_matrix = pd.concat([bounds_matrix, bounds])
        print(bounds)

bounds_matrix = bounds_matrix.iloc[9:,:].drop(0, axis = 1)

with open("../results/bounds_list_over_seeds_sim9", "wb") as output_file:
    pickle.dump(bounds_matrix, output_file)

# Create plots

Fixed sensitivity parameter for calculating bounds

In [ ]:
plot_location = "../results/plots/"

In [ ]:
experiment_list = ["sim10b", "sim10c", "sim10d", "sim10e"]      # Here choose experiments to be plotted
checkpoints = "_checkpoints.ckpt"
conf_level = np.array([1,2,3,4])

seed(1)

sensitivity_parameter_list = [1.2]
with torch.no_grad():   
    bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
helpers.plot_bounds_over_confounding(conf_level, bounds, oracle, path = plot_location + "bounds_over_confounding/bounds_over_confounding_sim10" + str(sensitivity_parameter_list) +".pdf")

sensitivity_parameter_list = [2.0]
with torch.no_grad():   
    bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
helpers.plot_bounds_over_confounding(conf_level, bounds, oracle, path = plot_location + "bounds_over_confounding/bounds_over_confounding_sim10" + str(sensitivity_parameter_list) +".pdf")

sensitivity_parameter_list = [5.0]
with torch.no_grad():   
    bounds, oracle = evaluation.calculate_bounds(label_col = "Y", sensitive_attributes = [0.0, 1.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
helpers.plot_bounds_over_confounding(conf_level, bounds, oracle, path = plot_location + "bounds_over_confounding/bounds_over_confounding_sim10" + str(sensitivity_parameter_list) +".pdf")

In [ ]:
experiment_list = ["sim_cont_b","sim_cont_c", "sim_cont_d", "sim_cont_e"]
checkpoints = "_checkpoints-v3.ckpt"
conf_level = np.array([1,2,3,4])

seed(2)

sensitivity_parameter_list = [2.0]
with torch.no_grad():   
    bounds, oracle = evaluation.calculate_bounds_continuous(label_col = "Y", sensitive_attributes = [1.0, 0.0], sensitivity_parameter_list = sensitivity_parameter_list, checkpoints=checkpoints, experiment_list = experiment_list)
helpers.plot_bounds_over_confounding_continuous(conf_level, bounds, path = plot_location + "bounds_over_confounding/bounds_over_confounding_sim_cont" + str(sensitivity_parameter_list) +".pdf")

### Dataset fairness (no classifier)

In [ ]:
seed(1)

conf_level = [1,2,3,4]
bounds, oracle, naive, tv = evaluation.caculate_data_fairness(sensitive_attributes = [0.0, 1.0], sensitivity_parameter=np.array([1.5, 5.0, 15.0]), experiment_list = ["sim9b","sim9c", "sim9d", "sim9e"])
helpers.plot_bounds_data(conf_level, oracle, naive, bounds, tv, path = plot_location + "bounds_data_sim9.pdf")

In [ ]:
seed(1)

conf_level = [1,2,3,4]
bounds, oracle, naive, tv = evaluation.caculate_data_fairness(sensitive_attributes = [0.0, 1.0], sensitivity_parameter=np.array([1.5, 2.0, 5.0]), experiment_list = ["sim10b","sim10c", "sim10d", "sim10e"])
helpers.plot_bounds_data(conf_level, oracle, naive, bounds, tv, path = plot_location + "bounds_data_sim10.pdf")

### Real-world case study

In [ ]:
seed(1)
# test data:
data_location = "../data/prison/prison_dataframe"    
experiment = "prison"

with open(data_location, "rb") as input:
    full_data = pickle.load(input)

full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)

test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
covariates = test_data.iloc[:, test_data.columns != "Y"]
covariates_white = covariates.loc[covariates["A"]==1.0].values
covariates_white = torch.tensor(covariates_white).float()
covariates_nonwhite = covariates.loc[covariates["A"]==0.0].values
covariates_nonwhite = torch.tensor(covariates_nonwhite).float()
covariates = torch.tensor(covariates.values).float()

# regressors

standard = StandardRegressor.load_from_checkpoint(model_location + "StandardRegressor_"+ experiment + "/StandardRegressor_"+ experiment + "_checkpoints-v1.ckpt")
fair = FairRegressor.load_from_checkpoint(model_location + "FairRegressor_"+ experiment + "/FairRegressor_"+ experiment + "_checkpoints.ckpt")

with torch.no_grad():
    out_standard= standard(covariates).detach().numpy().squeeze()
    out_fair= fair(covariates).detach().numpy().squeeze()

out_standard_df = pd.DataFrame({"Sentence length": out_standard})
out_standard_df["Race"] = test_data["A"].values
out_standard_df["Race"] = out_standard_df["Race"].replace(0.0, "non-white")
out_standard_df["Race"] = out_standard_df["Race"].replace(1.0, "white")

out_fair_df = pd.DataFrame({"Sentence length": out_fair})
out_fair_df["Race"] = test_data["A"].values
out_fair_df["Race"] = out_fair_df["Race"].replace(0.0, "non-white")
out_fair_df["Race"] = out_fair_df["Race"].replace(1.0, "white")

helpers.plot_sentence_distribution(out_standard_df, out_fair_df, path = plot_location + "prison_distribution.pdf")

### Performance change based on sensitivity parameter used for training

In [ ]:
seed(25)

experiment_list = ["prison_sensitivity_1.0", "prison_sensitivity_1.5", "prison_sensitivity_2.0", "prison_sensitivity_2.5", "prison_sensitivity_3.0", "prison_sensitivity_3.5", "prison_sensitivity_4"]
#checkpoint_list = ["_checkpoints.ckpt", "_checkpoints-v1.ckpt", "_checkpoints-v2.ckpt", "_checkpoints-v3.ckpt", "_checkpoints-v4.ckpt"]
checkpoint_list = ["_checkpoints-v5.ckpt", "_checkpoints-v6.ckpt", "_checkpoints-v7.ckpt", "_checkpoints-v8.ckpt", "_checkpoints-v9.ckpt"]
sensitivity_parameters = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]

performance_df = pd.DataFrame(columns=["sensitivity parameter", "mean", "std", "std_minus", "std_plus"])

ind = 0
for experiment in experiment_list:
    performance = []
    for checkpoints in checkpoint_list:
        with torch.no_grad():   
            data_location = "../data/prison/prison_dataframe"    
            fair = FairRegressor.load_from_checkpoint(model_location + "FairRegressor_"+ experiment + "/FairRegressor_"+ experiment + checkpoints)

            with open(data_location, "rb") as input:
                full_data = pickle.load(input)

            full_data = full_data.drop(["USE", "UIE", "UDE"],axis=1)
            mse = MeanSquaredError()

            test_data = full_data.iloc[int(0.8*full_data.shape[0]):,:]
            covariates = test_data.iloc[:, test_data.columns != "Y"].values
            covariates = torch.tensor(covariates).float()

            out = fair(covariates)
            result = mse(out.squeeze(), torch.tensor(test_data[label_col].to_numpy()).long())
            performance = np.append(performance, result)
           
    mean = np.mean(performance)
    std = np.std(performance)
    
    performance_df = pd.concat([performance_df, pd.DataFrame({"sensitivity parameter": [sensitivity_parameters[ind]], "mean": [mean], "std": [std], "std_minus": [mean-std], "std_plus": [mean+std]})])
    ind += 1

print(performance_df)

In [ ]:
performance_df["std_minus"] = (performance_df["std_minus"]/performance_df["mean"].to_numpy()[0])*100-100
performance_df["std_plus"] = (performance_df["std_plus"]/performance_df["mean"].to_numpy()[0])*100-100
performance_df["mean"] = (performance_df["mean"]/performance_df["mean"].to_numpy()[0])*100-100

helpers.plot_performance_over_sensitivity_param(performance_df, path = plot_location + "performance_over_sensitivity_param.pdf")